In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import matplotlib
import re
import numpy as np

In [2]:
import wget

url = 'https://files.grouplens.org/datasets/movielens/ml-25m.zip'
response = wget.download(url)

In [3]:
from zipfile import ZipFile

with ZipFile('ml-25m.zip', 'r') as zip:
    zip.printdir()
    zip.extractall() 

File Name                                             Modified             Size
ml-25m/                                        2019-11-21 16:41:38            0
ml-25m/tags.csv                                2019-11-21 15:55:50     38810332
ml-25m/links.csv                               2019-11-21 16:40:20      1368578
ml-25m/README.txt                              2019-11-21 16:41:38        10460
ml-25m/ratings.csv                             2019-11-21 16:26:42    678260987
ml-25m/genome-tags.csv                         2019-11-21 16:35:38        18103
ml-25m/genome-scores.csv                       2019-11-21 16:35:38    435164157
ml-25m/movies.csv                              2019-11-21 16:32:10      3038099


In [3]:
movieTG = pd.read_csv('./movies.csv', encoding = 'UTF-8')
movie_tag = pd.read_csv('./tags.csv', encoding = 'UTF-8')
movie_rating = pd.read_csv('./ratings.csv', encoding = 'UTF-8')
movie_url = pd.read_csv('./merge_all_3.csv', encoding = 'UTF-8')
movieTG.head()
# movieRT.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# movie_rating.sort_values(by='movieId')
movie_rating

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [5]:
movie_rating.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [5]:
# movie_rating.head(20)
import math

def change_to_year(x):
    x_to_year= ((((x/60)/60)/24)/30)/12
    year_full = math.floor(x_to_year)
    year_full_1970 = year_full+1970
    return year_full_1970

In [6]:
timestamp_year = []

for i in movie_rating['timestamp'][::]:
    x = change_to_year(i)
    timestamp_year.append(x)
    

In [7]:
# print(len(timestamp_year))
df_timestamp_year = pd.DataFrame(timestamp_year, columns = ['Timestamp_year'])
df_timestamp_year

,Timestamp_year
0,2006
1,2006
2,2006
3,2006
4,2006
...,...
25000090,2009
25000091,2009
25000092,2009
25000093,2009


In [8]:
movie_rating_wyear = pd.concat([movie_rating, df_timestamp_year], axis = 1)
movie_rating_wyear

,userId,movieId,rating,timestamp,Timestamp_year
0,1,296,5.0,1147880044,2006
1,1,306,3.5,1147868817,2006
2,1,307,5.0,1147868828,2006
3,1,665,5.0,1147878820,2006
4,1,899,3.5,1147868510,2006
...,...,...,...,...,...
25000090,162541,50872,4.5,1240953372,2009
25000091,162541,55768,2.5,1240951998,2009
25000092,162541,56176,2.0,1240950697,2009
25000093,162541,58559,4.0,1240953434,2009


In [9]:
group_by_id = movie_rating_wyear.groupby(['movieId','Timestamp_year']).size().reset_index(name = 'counts')
# group_by_id = group_by_id.get_group('Timestamp_year').size().reset_index(name='counts')
group_by_id

,movieId,Timestamp_year,counts
0,1,1996,3561
1,1,1997,8219
2,1,1998,1051
3,1,1999,519
4,1,2000,3990
...,...,...,...
333870,209157,2020,1
333871,209159,2020,1
333872,209163,2020,1
333873,209169,2020,1


In [10]:
df_pv = group_by_id.pivot(index="movieId", columns="Timestamp_year", values="counts")
df_pv

Timestamp_year,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3561.0,8219.0,1051.0,519.0,3990.0,2946.0,1553.0,1105.0,1629.0,...,1739.0,1269.0,1154.0,888.0,1105.0,4952.0,3058.0,2167.0,1918.0,1010.0
2,NaN,2938.0,3468.0,347.0,316.0,1341.0,1011.0,550.0,491.0,763.0,...,734.0,506.0,360.0,276.0,347.0,1218.0,1107.0,995.0,842.0,333.0
3,NaN,665.0,5368.0,310.0,153.0,641.0,646.0,251.0,192.0,333.0,...,99.0,152.0,95.0,30.0,45.0,127.0,129.0,88.0,78.0,30.0
4,NaN,639.0,648.0,71.0,60.0,331.0,215.0,78.0,90.0,73.0,...,7.0,7.0,12.0,5.0,11.0,27.0,23.0,15.0,9.0,6.0
5,NaN,1172.0,5103.0,259.0,98.0,459.0,385.0,210.0,187.0,280.0,...,96.0,195.0,213.0,149.0,119.0,290.0,263.0,204.0,143.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
209159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
209163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [11]:
import numpy as np
df_pv_sum = df_pv[::].cumsum(axis = 1)
df_pv_sum

Timestamp_year,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3561.0,11780.0,12831.0,13350.0,17340.0,20286.0,21839.0,22944.0,24573.0,...,39788.0,41057.0,42211.0,43099.0,44204.0,49156.0,52214.0,54381.0,56299.0,57309.0
2,NaN,2938.0,6406.0,6753.0,7069.0,8410.0,9421.0,9971.0,10462.0,11225.0,...,18244.0,18750.0,19110.0,19386.0,19733.0,20951.0,22058.0,23053.0,23895.0,24228.0
3,NaN,665.0,6033.0,6343.0,6496.0,7137.0,7783.0,8034.0,8226.0,8559.0,...,11030.0,11182.0,11277.0,11307.0,11352.0,11479.0,11608.0,11696.0,11774.0,11804.0
4,NaN,639.0,1287.0,1358.0,1418.0,1749.0,1964.0,2042.0,2132.0,2205.0,...,2408.0,2415.0,2427.0,2432.0,2443.0,2470.0,2493.0,2508.0,2517.0,2523.0
5,NaN,1172.0,6275.0,6534.0,6632.0,7091.0,7476.0,7686.0,7873.0,8153.0,...,10101.0,10296.0,10509.0,10658.0,10777.0,11067.0,11330.0,11534.0,11677.0,11714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
209159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
209163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [12]:
movie_id_title = movie_url[['movieId', 'title', 'poster_url']]
movie_race_r = movie_id_title.merge(df_pv_sum, on = 'movieId', how = 'right')
movie_race_r

,movieId,title,poster_url,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,Toy Story,https://m.media-amazon.com/images/M/MV5BMDU2ZW...,NaN,3561.0,11780.0,12831.0,13350.0,17340.0,20286.0,...,39788.0,41057.0,42211.0,43099.0,44204.0,49156.0,52214.0,54381.0,56299.0,57309.0
1,2,Jumanji,https://m.media-amazon.com/images/M/MV5BZTk2Zm...,NaN,2938.0,6406.0,6753.0,7069.0,8410.0,9421.0,...,18244.0,18750.0,19110.0,19386.0,19733.0,20951.0,22058.0,23053.0,23895.0,24228.0
2,3,Grumpier Old Men,https://m.media-amazon.com/images/M/MV5BMjQxM2...,NaN,665.0,6033.0,6343.0,6496.0,7137.0,7783.0,...,11030.0,11182.0,11277.0,11307.0,11352.0,11479.0,11608.0,11696.0,11774.0,11804.0
3,4,Waiting to Exhale,https://m.media-amazon.com/images/M/MV5BYzcyMD...,NaN,639.0,1287.0,1358.0,1418.0,1749.0,1964.0,...,2408.0,2415.0,2427.0,2432.0,2443.0,2470.0,2493.0,2508.0,2517.0,2523.0
4,5,Father of the Bride Part II,https://m.media-amazon.com/images/M/MV5BOTEyNz...,NaN,1172.0,6275.0,6534.0,6632.0,7091.0,7476.0,...,10101.0,10296.0,10509.0,10658.0,10777.0,11067.0,11330.0,11534.0,11677.0,11714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59042,209157,Wij,https://m.media-amazon.com/images/M/MV5BZjRkNz...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
59043,209159,Janela da Alma,https://m.media-amazon.com/images/M/MV5BMjE5Mj...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
59044,209163,Rossz versek,https://m.media-amazon.com/images/M/MV5BNTgxMz...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
59045,209169,A Girl Thing,https://m.media-amazon.com/images/M/MV5BMjE1MD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [13]:
movie_race_r.to_csv('movie_race_r.csv', index = False)

In [47]:
movie_id_title = movie_url[['movieId', 'title', 'poster_url']]
movie_race = movie_id_title.merge(df_pv_sum, on = 'movieId', how = 'right')
del movie_race['movieId']
movie_race

,title,poster_url,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Toy Story,https://m.media-amazon.com/images/M/MV5BMDU2ZW...,NaN,3561.0,11780.0,12831.0,13350.0,17340.0,20286.0,21839.0,...,39788.0,41057.0,42211.0,43099.0,44204.0,49156.0,52214.0,54381.0,56299.0,57309.0
1,Jumanji,https://m.media-amazon.com/images/M/MV5BZTk2Zm...,NaN,2938.0,6406.0,6753.0,7069.0,8410.0,9421.0,9971.0,...,18244.0,18750.0,19110.0,19386.0,19733.0,20951.0,22058.0,23053.0,23895.0,24228.0
2,Grumpier Old Men,https://m.media-amazon.com/images/M/MV5BMjQxM2...,NaN,665.0,6033.0,6343.0,6496.0,7137.0,7783.0,8034.0,...,11030.0,11182.0,11277.0,11307.0,11352.0,11479.0,11608.0,11696.0,11774.0,11804.0
3,Waiting to Exhale,https://m.media-amazon.com/images/M/MV5BYzcyMD...,NaN,639.0,1287.0,1358.0,1418.0,1749.0,1964.0,2042.0,...,2408.0,2415.0,2427.0,2432.0,2443.0,2470.0,2493.0,2508.0,2517.0,2523.0
4,Father of the Bride Part II,https://m.media-amazon.com/images/M/MV5BOTEyNz...,NaN,1172.0,6275.0,6534.0,6632.0,7091.0,7476.0,7686.0,...,10101.0,10296.0,10509.0,10658.0,10777.0,11067.0,11330.0,11534.0,11677.0,11714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59042,Wij,https://m.media-amazon.com/images/M/MV5BZjRkNz...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
59043,Janela da Alma,https://m.media-amazon.com/images/M/MV5BMjE5Mj...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
59044,Rossz versek,https://m.media-amazon.com/images/M/MV5BNTgxMz...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
59045,A Girl Thing,https://m.media-amazon.com/images/M/MV5BMjE1MD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [49]:
movie_race.to_csv('movie_race.csv', index = False, encoding = 'UTF-8')

In [ ]:
movie_id_title = movie_url[['movieId', 'title', 'poster_url']]
movie_race = movie_id_title.merge(movie_rating_wyear, on = 'movieId', how = 'right')
movie_race

,movieId,title,poster_url,Timestamp_year
0,296,Pulp Fiction,https://m.media-amazon.com/images/M/MV5BNGNhMD...,2006
1,306,Trois couleurs: Rouge,https://m.media-amazon.com/images/M/MV5BYTg1Mm...,2006
2,307,Trois couleurs: Bleu,https://m.media-amazon.com/images/M/MV5BZWU5NW...,2006
3,665,Underground,https://m.media-amazon.com/images/M/MV5BNzI4YT...,2006
4,899,Singin&apos; in the Rain,https://m.media-amazon.com/images/M/MV5BZDRjNG...,2006
...,...,...,...,...
25000090,50872,Ratatouille,https://m.media-amazon.com/images/M/MV5BMTMzOD...,2009
25000091,55768,Bee Movie,https://m.media-amazon.com/images/M/MV5BMjE1MD...,2009
25000092,56176,Alvin and the Chipmunks,https://m.media-amazon.com/images/M/MV5BMjdmNW...,2009
25000093,58559,The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2009


In [25]:
# movie_rating['rating'].describe()
movie_rating['rating'].value_counts()

4.0    6639798
3.0    4896928
5.0    3612474
3.5    3177318
4.5    2200539
2.0    1640868
2.5    1262797
1.0     776815
1.5     399490
0.5     393068
Name: rating, dtype: int64

In [40]:
genres = movie_genres[0].unique()
genres=genres[genres!=np.array(None)]
genres=np.delete(genres,19)
genres

array(['Adventure', 'Comedy', 'Action', 'Drama', 'Crime', 'Children',
       'Mystery', 'Documentary', 'Animation', 'Thriller', 'Horror',
       'Fantasy', 'Western', 'Film-Noir', 'Romance', 'War', 'Sci-Fi',
       'Musical', 'IMAX'], dtype=object)

In [41]:
avg_ratings=movie_rating.groupby('movieId', as_index=False).mean()
del avg_ratings['userId']
del avg_ratings['timestamp']
avg_ratings.head()

,movieId,rating
0,1,3.893708
1,2,3.251527
2,3,3.142028
3,4,2.853547
4,5,3.058434


In [42]:
count_ratings=movie_rating.groupby('movieId', as_index=False).count()
del count_ratings['userId']
del count_ratings['timestamp']
count_ratings.columns.values[1]='count'
count_ratings.head()

,movieId,count
0,1,57309
1,2,24228
2,3,11804
3,4,2523
4,5,11714


In [ ]:
final=movieTG.merge(avg_ratings, on='movieId', how='inner')
final=final.merge(count_ratings, on='movieId', how='inner')
final.sort_value(by = 'count')

In [51]:
final.sort_values(by = ['count'])

,movieId,title,genres,rating,count
29523,136782,The Girl is in Trouble (2015),Thriller,5.000000,1
34959,151011,The Secret of the Magic Gourd (2007),Children|Drama,2.500000,1
34958,151001,Justice for Natalee Holloway (2011),Crime|Drama|Mystery|Thriller,1.000000,1
34951,150962,Little Big Shot (1935),Comedy|Crime|Drama,3.000000,1
34947,150950,Harlem Rides the Range (1939),Western,1.000000,1
...,...,...,...,...,...
2480,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.154099,72674
585,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.151342,74127
292,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.188912,79672
314,318,"Shawshank Redemption, The (1994)",Crime|Drama,4.413576,81482
